<a href="https://colab.research.google.com/github/mikexcohen/Calculus_book/blob/main/exercises/ch17_integrationGeometry_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Calculus unraveled: Intuition, Proofs, and Python**
### Mike X Cohen (sincxpress.com)
#### https://github.com/mikexcohen/calculus_book
#### Code for Chapter 17 (Integration applications in geometry)

---

# About this code file:

### This notebook contains full code solutions to the exercises in this book chapter. There are many correct ways to solve the exercises; this notebook provides *a* solution, not *THE* solution. Please use this code as a starting point to continue exploring and experimenting with calculus concepts and visualizations.

## **Using the code without the book may lead to confusion or errors.**

#### This code was written in google-colab. The notebook may require some modifications if you use a different IDE.

In [ ]:
# import libraries and define global settings
import numpy as np
import sympy as sym
import scipy.integrate as spi
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from IPython.display import Math

# define global figure properties used for publication
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg') # display figures in vector format
plt.rcParams.update({'font.size':14,             # font size
                     'savefig.dpi':300,          # output resolution
                     'axes.titlelocation':'left',# title location
                     'axes.spines.right':False,  # remove axis bounding box
                     'axes.spines.top':False,    # remove axis bounding box
                     'lines.linewidth':2         # increase default line thickness
                     })

# Exercise 17.1: Confirmation in scipy

In [ ]:
# I used example 1 (Figure 17.3)

In [ ]:
# functions
x  = sym.symbols('x')
fx = sym.exp(-x**2)
gx = x**2

# bounds
a,b = 0,1

# numerical evaluation
xx = np.linspace(-.2,1.2,499)
y_fx = np.array([fx.subs(x,xi) for xi in xx],dtype=float)
y_gx = np.array([gx.subs(x,xi) for xi in xx],dtype=float)
xInBnds = (xx>=a) & (xx<=b)

# draw the figure
_,axs = plt.subplots(1,2,figsize=(12,3.5))
axs[0].plot(xx,y_fx,'k',label=r'$f(x) = %s$' %sym.latex(fx))
axs[0].plot(xx,y_gx,'--',color=[.5,.5,.5],label=r'$g(x) = %s$' %sym.latex(gx))
axs[0].fill_between(xx[xInBnds],y_fx[xInBnds],y_gx[xInBnds],color='k',alpha=.2,label='Area')

axs[0].legend(loc=(.45,.6))
axs[0].set(xlim=xx[[0,-1]],xlabel='x',ylabel=r'$y$',title=r'$\bf{A}$)  Both functions')

axs[1].plot(xx,y_fx-y_gx,'k',label=r'$f(x) - g(x)$')
axs[1].fill_between(xx[xInBnds],y_fx[xInBnds]-y_gx[xInBnds],color='k',alpha=.2,label='Area')
axs[1].axhline(0,linestyle=':',color=[.8,.8,.8],zorder=-4)

axs[1].legend()
axs[1].set(xlim=xx[[0,-1]],xlabel='x',ylabel=r'$y_f-y_g$',title=r'$\bf{B}$)  Function difference')

# and save
plt.tight_layout()
plt.show()

In [ ]:
# net area
netArea = spi.trapezoid(y_fx[xInBnds]-y_gx[xInBnds],dx=xx[1]-xx[0])

# total area
totArea = spi.trapezoid(abs(y_fx[xInBnds]-y_gx[xInBnds]),dx=xx[1]-xx[0])

print(f'Net area: {netArea:.4f}')
print(f'Total area: {totArea:.4f}')

# Exercise 17.2: FTC-2

In [ ]:
# setup the equations and antiderivatives
x = sym.symbols('x')

fx = sym.exp(-x**2)
fi = sym.integrate(fx,x)

gx = x**2
gi = sym.integrate(gx,x)

hx = fx-gx
hi = sym.integrate(hx,x)

# integrations
a,b = 0,1
fi_a = fi.subs(x,a)
fi_b = fi.subs(x,b)

gi_a = gi.subs(x,a)
gi_b = gi.subs(x,b)

hi_a = hi.subs(x,a)
hi_b = hi.subs(x,b)

# print the results
display(Math('\\qquad\\qquad\\qquad\;\; H\\left(%s\\right)-H\\left(%s\\right) \;=\; %s-%s \quad\;=\; %s'
             %(b,a,sym.latex(hi_b),sym.latex(hi_a),sym.latex((hi_b-hi_a)))))
print('')
display(Math('F\\left(%s\\right) + G\\left(%s\\right) - G\\left(%s\\right) - F\\left(%s\\right) \;=\; %s+%s-%s-%s \;=\; %s'
             %(b,a,b,a,  sym.latex(fi_b),sym.latex(gi_a),sym.latex(gi_b),sym.latex(fi_a),
               sym.latex( fi_b+gi_a-gi_b-fi_a ))))

# Exercise 17.3: Lips

In [ ]:
l = sym.symbols('lambda',real=True)

f = sym.sqrt(sym.Abs(l))+3
g = l**2+sym.exp(sym.sympify(1))/2
h = sym.Abs(l)**sym.exp(sym.sympify(1))

# print the functions
display(Math('f(\\lambda) = %s' %sym.latex(f))), print('')
display(Math('g(\\lambda) = %s' %sym.latex(g))), print('')
display(Math('h(\\lambda) = %s' %sym.latex(h)))

In [ ]:
### find the intersection points

# solvable:
# fg_intersection = sym.solve(f-g)
# fg_intersection[0].evalf()

# uh oh
# gh_intersection = sym.solve(g-h)
# gh_intersection[0].evalf()

In [ ]:
# need to approximate the intersection
f_lam = sym.lambdify(l,f)
g_lam = sym.lambdify(l,g)
h_lam = sym.lambdify(l,h)

# numerical evaluations
ll = np.linspace(-2,2,1001)
f_num = f_lam(ll)
g_num = g_lam(ll)
h_num = h_lam(ll)

# get the intersections from their differences
zeroidx = np.argmin(abs(ll))
fg_intersect = [ np.argmin(abs(f_num-g_num)[:zeroidx]),
                 np.argmin(abs(f_num-g_num)[zeroidx:])+zeroidx ]
gh_intersect = [ np.argmin(abs(g_num-h_num)[:zeroidx]),
                 np.argmin(abs(g_num-h_num)[zeroidx:])+zeroidx ]

# print the results
print(ll[fg_intersect])
print(ll[gh_intersect])

In [ ]:
# intermediate plot for intersection points
plt.figure(figsize=(4,4))
plt.plot(ll,abs(f_num-g_num),'k',label='|f-g|')
plt.plot(ll,abs(g_num-h_num),'--',color=[.7,.7,.7],label='|f-h|')
plt.axhline(0,linestyle=':',color=[.8,.8,.8])

plt.plot(ll[fg_intersect],[0,0],'ko',markerfacecolor='w')

plt.gca().set(xlim=ll[[0,-1]],xlabel=r'$\lambda$',ylabel='abs(function differences)')
plt.legend()

plt.tight_layout()
plt.savefig('intGeom_ex3b.png')
plt.show()

In [ ]:
# calculate areas using scipy

# points between intersections
l4fg = (ll>=ll[fg_intersect[0]]) & (ll<=ll[fg_intersect[1]])
l4gh = (ll>=ll[gh_intersect[0]]) & (ll<=ll[gh_intersect[1]])

# area estimates
area_fg = spi.trapezoid(f_num[l4fg]-g_num[l4fg],dx=ll[1]-ll[0])
area_gh = spi.trapezoid(g_num[l4gh]-h_num[l4gh],dx=ll[1]-ll[0])

# print the results
display(Math('%s \\approx %.2f' %(sym.latex(sym.Integral(f-g,(l,ll[fg_intersect[0]],ll[fg_intersect[1]]))),area_fg))), print('')
display(Math('%s \\approx %.2f' %(sym.latex(sym.Integral(g-h,(l,ll[gh_intersect[0]],ll[gh_intersect[1]]))),area_gh)))

In [ ]:
# final plot
plt.figure(figsize=(10,4))

# the functions
plt.plot(ll,f_num,'k',label=r'$f(\lambda) = %s$' %sym.latex(f))
plt.plot(ll,g_num,'--',color=[.7,.7,.7],label=r'$g(\lambda) = %s$' %sym.latex(g))
plt.plot(ll,h_num,':',color=[.4,.4,.4],label=r'$h(\lambda) = %s$' %sym.latex(h))

# the areas (book is printed with grayscale, but I like the commented colors)
plt.fill_between(ll[l4fg],f_num[l4fg],g_num[l4fg],color='k',alpha=.5)#np.array([217,101,7])/255)
plt.fill_between(ll[l4gh],g_num[l4gh],h_num[l4gh],color='k',alpha=.2)#np.array([78,25,107])/255)


# the final touches
plt.legend()
plt.gca().set(xlim=ll[[0,-1]],xlabel=r'$\lambda$',ylabel=r'$y=f(\lambda)$ or $g(\lambda)$ or $h(\lambda)$')

plt.tight_layout()
plt.savefig('intGeom_ex3a.png')
plt.show()

# Exercise 17.4:

In [ ]:
# variable, function, integral
x = sym.symbols('x')
fx = sym.log(x)
gx = sym.integrate(fx)

# print to confirm
display(Math('%s \;=\; %s' %(sym.latex(sym.Integral(fx)),sym.latex(gx))))

In [ ]:
# find the intersection points
sym.solve(fx-gx)

In [ ]:
# approximate the intersection points
# lambdify the functions
fx_l = sym.lambdify(x,fx)
gx_l = sym.lambdify(x,gx)
diff_l = sym.lambdify(x,fx-gx)

# evaluate the functions
xx = np.linspace(.01,4.5,323)

# find points closest to zero
peekz = find_peaks(-abs(diff_l(xx)))[0]
pinchPoints = xx[peekz]

# exact definite integral using sympy
area = sym.integrate(fx-gx,(x,pinchPoints[0],pinchPoints[1]))

# optional plot
plt.plot(xx,abs(diff_l(xx)))
plt.plot(pinchPoints,abs(diff_l(pinchPoints)),'mo')
plt.ylim(0,1.5)
plt.xlabel('x')
plt.ylabel('|f-g|')
plt.show()

In [ ]:
# and draw
plt.figure(figsize=(10,4))

# the lines
plt.plot(xx,fx_l(xx),'k',label=r'$f(x) = %s$' %sym.latex(fx))
plt.plot(xx,gx_l(xx),'--',color=[.7,.7,.7],label=r'$g(x) = %s$' %sym.latex(gx))

# the area
x4area = xx[(xx>=pinchPoints[0]) & (xx<=pinchPoints[1])]
plt.fill_between(x4area,fx_l(x4area),gx_l(x4area),color='k',alpha=.2)

# the intersection points
plt.plot(pinchPoints,fx_l(pinchPoints),'ko',markersize=8,markerfacecolor='w')

# the pretty stuff
plt.gca().set(xlabel='x',ylabel='y',xlim=xx[[0,-1]],ylim=[-1.5,2])
plt.title(fr'Area = {area.evalf():.3f} $(au)^2$',loc='center')
plt.legend()

# the export
plt.tight_layout()
plt.savefig('intGeom_ex4.png')
plt.show()

# Exercise 17.5: Parametric analysis of log parameter

In [ ]:
# need to extend the x-axis
xx = np.linspace(.01,8,123)

aParams = np.linspace(.4,5,33)
areas = np.zeros(len(aParams))

_,axs = plt.subplots(1,3,figsize=(14,4))

for idx,a in enumerate(aParams):

  # the function, its antiderivative, and their difference as lambda functions
  fx = sym.lambdify(x,sym.log(a*x))
  fi = sym.lambdify(x,sym.integrate(sym.log(a*x)))
  fdiff = sym.lambdify(x,sym.log(a*x)-sym.integrate(sym.log(a*x)))

  # plot the two functions
  axs[0].plot(xx,fx(xx),linewidth=1,color=np.full(3,idx/(len(aParams)+3)))
  axs[0].plot(xx,fi(xx),'--',linewidth=1,color=np.full(3,idx/(len(aParams)+3)))

  # find integration bounds and calculate the area
  peekz = find_peaks(-np.abs(fdiff(xx)))[0]
  areas[idx],_ = spi.quad(fdiff,xx[peekz[0]],xx[peekz[1]])

  # plot the first and final lines for 'a'
  if (idx==0) or (idx==len(aParams)-1):
    axs[1].plot(xx,fx(xx),color=np.full(3,idx/(len(aParams)+3)),label=fr'$\alpha$ = {a:.3f}')
    axs[1].plot(xx,fi(xx),'--',color=np.full(3,idx/(len(aParams)+3)))

    # patch
    x4area = xx[(xx>=xx[peekz[0]]) & (xx<=xx[peekz[1]])]
    axs[1].fill_between(x4area,fi(x4area),fx(x4area),color='k',alpha=.3)


# plot the area as a function of the 'a' parameter
axs[2].plot(aParams,areas,'ks-',markersize=8,markerfacecolor=[.7,.7,.7])

# some final adjustments
axs[0].set(xlim=xx[[0,-1]],xlabel='x',ylabel='y',title=r'$\bf{A}$)  The functions')
axs[1].set(xlim=xx[[0,-1]],xlabel='x',ylabel='y',ylim=[-7,5],title=r'$\bf{B}$)  Two examples')
axs[1].legend()
axs[2].set(xlabel=r"$\alpha$ parameter in $f(x)=\ln(\alpha x)$",ylabel='Area',title=r'$\bf{C}$)  Area between intersections')

plt.tight_layout()
plt.savefig('intGeom_ex5.png')
plt.show()

# Exercise 17.6: Parametric curves

In [ ]:
# organized as a list of dictionaries

functions = [
  { # panel A
    't': np.linspace(-2*np.pi, 2*np.pi, 301),
    'x': lambda t: np.sin(2*t) + np.sin(t),
    'y': lambda t: 2*np.sin(3*t)
  },
  { # panel B
    't': np.linspace(0, 2*np.pi, 101),
    'x': lambda t: 3*np.cos(t) + np.cos(3*t),
    'y': lambda t: 3*np.sin(t) - np.sin(4*t)
  },
  { # panel C
    't': np.linspace(0, 12*np.pi, 1001),
    'x': lambda t: np.sin(t)*(np.exp(np.cos(t)) - 2*np.cos(4*t) - np.sin(t/12)**5),
    'y': lambda t: np.cos(t)*(np.exp(np.cos(t)) - 2*np.cos(4*t) - np.sin(t/12)**5)
  },
  { # panel D
    't': np.linspace(0, 12*np.pi, 1001),
    'x': lambda t: np.sqrt(3)*np.cos(2*t) - np.cos(10*t)*np.sin(20*t),
    'y': lambda t: -np.sqrt(2)*np.sin(2*t) - np.sin(10*t)*np.sin(20*t)
  },
  { # panel E
    't': np.linspace(0, 2*np.pi, 5001),
    'x': lambda t: np.cos(t) + np.cos(52*t)/2 + np.sin(25*t)/3,
    'y': lambda t: np.sin(t) + np.sin(52*t)/2 + np.cos(25*t)/3
  },
  { # panel F
    't': np.linspace(-2*np.pi, 2*np.pi, 10001),
    'x': lambda t: 2.5*np.sin(np.sin(-5*t)) * np.cos(9.844*t)**2,
    'y': lambda t: 2.5*np.sin(-5*t)**2 * 2**(np.cos(np.cos(9.844*t)))
  }
]

# Loop over functions in groups of 3
_,axs = plt.subplots(2,3,figsize=(12,6))
axs = axs.flatten()
titles = 'ABCDEF'

for i in range(len(functions)):

  # extract the data
  t = functions[i]['t']
  x = functions[i]['x'](t)
  y = functions[i]['y'](t)

  # and plot
  axs[i].plot(x,y,'k')
  axs[i].axis('off')
  axs[i].set_title(fr'$\bf{{{titles[i]}}}$)')


plt.tight_layout()
plt.savefig('intGeom_ex6a.png')
plt.show()

In [ ]:
# extract the data
t = functions[5]['t']
x = functions[5]['x'](t)
y = functions[5]['y'](t)

plt.figure(figsize=(4,4))
plt.scatter(x,y,s=5,c=t,cmap='gray')
plt.axis('off')
plt.tight_layout()
plt.savefig('intGeom_ex6b.png')
plt.show()

# Exercise 17.7

In [ ]:
a = 0
b = 12*np.pi
delta_t = .001

# extract the data
t = np.arange(a,b+delta_t,step=delta_t)
x = functions[2]['x'](t)
y = functions[2]['y'](t)

# derivatives and segment lengths
dx = x[1:]-x[:-1]
dy = y[1:]-y[:-1]
segLengths = np.sqrt( dx**2 + dy**2 )

# curve length as direct implementation in numpy
print(f'Total curve length for n = {len(t)}: {np.sum(segLengths)}')


# plot
_,axs = plt.subplots(1,2,figsize=(11,4))
axs[0].plot(x,y,'k',linewidth=1)
axs[0].set(xlabel='x',ylabel='y',title=r'$\bf{A}$)  The butterfly ($\Delta t = %g$, $L=%g$)' %(delta_t,np.round(np.sum(segLengths),2)))

axs[1].hist(segLengths,30,facecolor='k',alpha=.4,edgecolor='k')
axs[1].set(ylabel='Count',xlabel='Length (a.u.)',title=r'$\bf{B}$)  Histogram of segment lengths')


plt.tight_layout()
plt.savefig('intGeom_ex7.png')
plt.show()

# Exercise 17.8

In [ ]:
delta_ts = np.linspace(.0001,1,30)

arcLengths  = np.zeros(len(delta_ts))
arcLengthsn = np.zeros(len(delta_ts))

# create a figure
_,axs = plt.subplots(1,2,figsize=(12,5))

# loop over the values of \Delta t
for idx,dt in enumerate(delta_ts):

  t = np.arange(0,12*np.pi+dt,step=dt)
  x = functions[2]['x'](t)
  y = functions[2]['y'](t)

  # differences
  dx = x[1:]-x[:-1]
  dy = y[1:]-y[:-1]

  # all-numpy solution
  arcLengthsn[idx] = np.sum( np.sqrt( dx**2 + dy**2 ) )

  # Simpson's rule
  integrand = np.sqrt((dx/dt)**2 + (dy/dt)**2)
  arcLengths[idx] = spi.simpson(integrand,dx=dt)


  if idx==0:
    axs[0].plot(x,y,'k',linewidth=1,label=r'$\Delta t$ = %g'%dt)
  elif idx==(len(delta_ts)-1):
    axs[0].plot(x,y,'s-',linewidth=1,color=[.7,.7,.7],markerfacecolor=[.5,.5,.5],label=r'$\Delta t$ = %g'%dt)


axs[0].set(xlabel='$x$',ylabel='$y$',title=r'$\bf{A}$)  Example curves for two $\Delta t$')
axs[0].legend()

axs[1].plot(delta_ts,arcLengthsn,'ks',markersize=11,markerfacecolor=[.3,.3,.3],label='Numpy')
axs[1].plot(delta_ts,arcLengths ,'ko',markersize=11,markerfacecolor=[.7,.7,.7],alpha=.6,label="Simpson's")
axs[1].invert_xaxis()
axs[1].legend()
axs[1].set(xlabel=r'$\Delta t$',ylabel='Curve length',title=r'$\bf{B}$)  Curve length by $\Delta t$')

plt.tight_layout()
plt.savefig('intGeom_ex8.png')
plt.show()

# Exercise 17.9: Folium of Descartes

In [ ]:
# the folium in polar coords
th = np.linspace(0,2*np.pi,1001)
r_num = 3 * np.sin(th) * np.cos(th)
r_den = np.sin(th)**3 + np.cos(th)**3
r = r_num / r_den

# and in xy coords
t = np.linspace(-10,10,900)
x = 3*t / (1+t**3)
y = 3*t**2 / (1+t**3)


# the plot
colors = np.linspace([0,0,0],[.9,.9,.9],len(t))
plt.figure(figsize=(9,4))
ax1 = plt.subplot(1,2,1,projection='polar')
ax1.plot(th,r,'k')
ax1.set(rlim=[0,2.5],rticks=[1,2],title=r'$\bf{A}$)  Polar coords')
ax1.grid(color=[.9,.9,.9])

ax2 = plt.subplot(1,2,2)
ax2.scatter(x,y,s=20,c=colors)
ax2.set(xlim=[-5,5],ylim=[-5,5],xlabel='x',ylabel='y',title=r'$\bf{B}$)  Rectanguar coords')
ax2.set_aspect('equal')

plt.tight_layout()
plt.savefig('intGeom_ex9.png')
plt.show()

# Exercise 17.10: Measuring the Folium

In [ ]:
np.nansum(segLengths)

In [ ]:
# recalculate x and y
x = 3*t / (1+t**3)
y = 3*t**2 / (1+t**3)

# reject data points based on extreme xy values
outliers = np.sqrt( x**2 + y**2 ) > 5
x[outliers] = np.nan
y[outliers] = np.nan

# derivatives and segment lengths
dx = np.append(0, x[1:]-x[:-1])
dy = np.append(0, y[1:]-y[:-1])
dt = t[1] - t[0]
segLengths = np.sqrt( dx**2 + dy**2 )

# curve length via numpy calculation
curveLengthn = np.nansum(segLengths)

# curve length via Simpson's rule
integrand = np.sqrt((dx/dt)**2 + (dy/dt)**2)
integrand = integrand[~np.isnan(integrand)]
curveLength = spi.simpson(integrand,dx=dt)


# plot
colors = np.linspace([0,0,0],[.9,.9,.9],len(t))
_,axs = plt.subplots(2,2,figsize=(12,7))
axs[0,0].scatter(x,y,s=20,c=colors)
axs[0,0].set(xlim=[-5,5],ylim=[-5,5],xlabel='$x$',ylabel='$y$',title=r'$\bf{A}$)  The Folium')

axs[0,1].plot(t,x,'k',label='$x$')
axs[0,1].plot(t,y,color=[.7,.7,.7],label='$y$')
axs[0,1].set(xlim=t[[0,-1]],ylim=[-5,5],xlabel='$t$',ylabel='$x$ or $y$',title=r'$\bf{B}$)  The functions x(t) and y(t)')
axs[0,1].legend()

axs[1,0].scatter(t,segLengths,s=20,c=colors)
axs[1,0].set(xlim=t[[0,-1]],xlabel='$t$',ylim=[0,.1],ylabel='length (a.u.)',title=r'$\bf{C}$)  Segment lengths')

axs[1,1].scatter(t,np.nancumsum(segLengths),s=20,c=colors)
axs[1,1].set(xlim=t[[0,-1]],xlabel='$t$',ylabel='length (a.u.)',title=r'$\bf{D}$)  Cumulative lengths')

# print the curve lengths
print(f'(Simpson) Total curve length = {curveLength:.4f}')
print(f'(numpy)   Total curve length = {curveLengthn:.4f}')

plt.tight_layout()
plt.savefig('intGeom_ex10.png')
plt.show()

# Exercise 17.11

In [ ]:
### calculate the volume

# lambda-axis shift
shiftval = 1.71

# bounds
a,b = 0,2*shiftval
xx = np.linspace(a,b,500)

# functions
f = lambda u: np.sqrt(abs(u-shiftval)) + 3
h = lambda u: abs(u-shiftval)**np.exp(1)


# volume calculation
volume = np.pi * spi.simpson(np.abs(f(xx)**2-h(xx)**2),dx=xx[1]-xx[0])



### visualization

# get Y and Z coordinates for f(x) and h(x)
X,Theta = np.meshgrid(xx,np.linspace(0,2*np.pi,100))
Y_f = f(X) * np.cos(Theta)
Z_f = f(X) * np.sin(Theta)
Y_h = h(X) * np.cos(Theta)
Z_h = h(X) * np.sin(Theta)

# setup the figure with a 3D axis
fig = plt.figure(figsize=(12,5))
gs  = fig.add_gridspec(1,2,width_ratios=[2,3])
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1],projection='3d')

# plot the 2D shape to revolve
ax1.plot(xx,f(xx),'k',label=r'$f(\lambda)$')
ax1.plot(xx,h(xx),'--',color=[.7,.7,.7],label=r'$h(\lambda)$')
ax1.fill_between(xx,f(xx),h(xx),color='k',alpha=.2,label='Area to revolve')
ax1.set(xlabel=r'$\lambda$',ylabel=r'$y = f(\lambda)$ or $h(\lambda)$',xlim=xx[[0,-1]],title='2D area to revolve')
ax1.legend()

# create the surfaces
ax2.plot_surface(X, Y_f, Z_f, color='k', alpha=.3)
ax2.plot_surface(X, Y_h, Z_h, color='k', alpha=.6)
ax2.set(xlabel=r'$\lambda$',ylabel='Y',zlabel='Z',title=r'Volume $\approx$ $%.3f$ (au)$^3$' %volume)

plt.tight_layout()
plt.savefig('intGeom_ex11.png')
plt.show()

# Exercise 17.12: Ruben's faces

In [ ]:
# xy coordinates
y = np.array([1,0.981328,0.975104,0.973029,0.96888,0.960581,0.952282,0.943983,0.93361,0.925311,0.917012,0.906639,0.892116,0.875519,0.860996,0.844398,0.817427,0.804979,0.788382,0.746888,0.73029,0.717842,0.688797,0.6639,0.647303,0.624481,0.605809,0.589212,0.562241,0.545643,0.529046,0.506224,0.495851,0.481328,0.477178,0.462656,0.452282,0.43361,0.423237,0.410788,0.404564,0.39834,0.394191,0.390041,0.387967,0.383817,0.377593,0.363071,0.350622,0.342324,0.325726,0.311203,0.302905,0.275934,0.261411,0.246888,0.23029,0.211618,0.19917,0.190871,0.174274,0.16805,0.157676,0.149378,0.141079,0.13278,0.126556,0.116183,0.103734,0.0871369,0.0726141,0.060166,0.0435685,0.026971,0.0124481,0])
xR = np.array([1, 0.96094299, 0.91765732, 0.87437165, 0.80078537, 0.7574997 , 0.7272, 0.6969003 , 0.65794329, 0.62764268, 0.58868567, 0.55405732, 0.50644299, 0.46748597, 0.42852896, 0.38957104, 0.33762896, 0.31598567, 0.3029997 , 0.28135732, 0.2727    , 0.2727    , 0.28135732, 0.28568597, 0.27702866, 0.25971403, 0.22941433, 0.20344329, 0.13851433, 0.1168714 , 0.09522857, 0.0909    , 0.1168714 , 0.13851433, 0.14717146, 0.16448573, 0.1688143 , 0.14717146, 0.1298571 , 0.12552854, 0.13851433, 0.15150003, 0.16448573, 0.17314287, 0.17314287, 0.16448573, 0.15150003, 0.14717146, 0.16015716, 0.17747143, 0.20777104, 0.22508567, 0.22508567, 0.2120997 , 0.19911463, 0.1818    , 0.17314287, 0.1818    , 0.2120997 , 0.23374299, 0.32464299, 0.36792866, 0.44584268, 0.51077165, 0.56704329, 0.61898537, 0.6665997 , 0.70122896, 0.73585732, 0.77914299, 0.82242866, 0.8483997 , 0.87004299, 0.89601403, 0.91332866,0.93064329])
xL = np.array([-1, -0.96094299, -0.91765732, -0.87437165, -0.80078537,-0.7574997 , -0.7272 , -0.6969003 , -0.65794329, -0.62764268,-0.58868567, -0.55405732, -0.50644299, -0.46748597, -0.42852896,-0.38957104, -0.33762896, -0.31598567, -0.3029997 , -0.28135732,-0.2727    , -0.2727    , -0.28135732, -0.28568597, -0.27702866,-0.25971403, -0.22941433, -0.20344329, -0.13851433, -0.1168714 ,-0.09522857, -0.0909    , -0.1168714 , -0.13851433, -0.14717146,-0.16448573, -0.1688143 , -0.14717146, -0.1298571 , -0.12552854,-0.13851433, -0.15150003, -0.16448573, -0.17314287, -0.17314287,-0.16448573, -0.15150003, -0.14717146, -0.16015716, -0.17747143,-0.20777104, -0.22508567, -0.22508567, -0.2120997 , -0.19911463,-0.1818    , -0.17314287, -0.1818    , -0.2120997 , -0.23374299,-0.32464299, -0.36792866, -0.44584268, -0.51077165, -0.56704329,-0.61898537, -0.6665997 , -0.70122896, -0.73585732, -0.77914299,-0.82242866, -0.8483997 , -0.87004299, -0.89601403, -0.91332866,-0.93064329])

In [ ]:
# draw the graph
plt.figure(figsize=(8,8))
plt.plot(xL,y,'k-')
plt.plot(xR,y,'k-')
plt.fill_between(xR,y,color='k',alpha=.4)
plt.fill_between(xL,y,color='k',alpha=.4)

plt.tight_layout()
plt.savefig('intGeom_ex12.png')
plt.show()

In [ ]:
# HINT: flip the axes:
plt.figure(figsize=(10,8))
plt.plot(y,xL,'g.-',markersize=10)
plt.plot(y,xR,'r.-',markersize=10)

plt.fill_between(y,xR,xL,color='k',alpha=.4)
plt.fill_between(y,xL,-np.ones(len(y)),color='r',alpha=.4)
plt.fill_between(y,xR,np.ones(len(y)),color='b',alpha=.2)

plt.show()

In [ ]:
# now compute the area. consider each individual slice to be a trapezoid.
# area of a trapezoid is (a+b)/2*h
areaVase = 0
areaLeft = 0
areaRigt = 0

for idx in range(1,len(y)):

  # calculate each part of the area formula
  h = y[idx-1] - y[idx]
  a = xR[idx-1]- xL[idx-1]
  b = xR[idx]  - xL[idx]

  # area of the slice and add to the total
  areaVase += (a+b)/2 * h
  areaLeft += (xL[idx]+xL[idx-1] + 2)/2 * h
  areaRigt += (2 - xR[idx]-xR[idx-1])/2 * h

print(f'Total area of the vase       : {areaVase:.3f}')
print(f'Total area of the left face  : {areaLeft:.3f}')
print(f'Total area of the right face : {areaRigt:.3f}')
print(f'Sum of three areas           : {areaLeft+areaRigt+areaVase:.3f}')